In [33]:
import copy, sys, pandas as pd
import random
import time
import numpy as np

sys.path.append("../../")
sys.path.append("../../inferring")
sys.path.append("../../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS

from inferring.InferringDFA import InferringDFA

In [34]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [35]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                 cnt_with_advice,
                 target=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.cnt_with_advice = cnt_with_advice
        self.target = target

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_with_advice}"
        )
    def get_complete_description(self):
        return self.target.print_complete_description()
    
    def print_target(self):
        print(self.target.print_complete_description())

In [ ]:
number_of_itreations = 5

#  Fixed random seeds for reproducibility
seeds = [i for i in range(number_of_itreations)]

# The alphabet of the DFA
input_signs = ["a", "b", "c"]

# The bound on the number of states
max_number_of_states = 50

# The list to aggregate the results
results = []

i = 0
while i < number_of_itreations:
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    # Create two random DFAs
    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )
    dfa2.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs,
    )

    # Run learning of the language of DFA to minimize dfa1 and dfa2
    d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    while d1.Q < 2 or d2.Q < 2:
        # Create two random DFAs
        dfa1 = DFA()
        dfa2 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs,
        )

        # Run learning of the language of DFA to minimize dfa1 and dfa2
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # Create a convolution DFA based on d1 and d2
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2)

    # Learn conv_dfa with advice
    start_t = time.time()
    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(
        target=conv_dfa, advice_system=SRS(), check_consistency=True
    )
    end_t = time.time()

    print("ex time with AS: ", end_t - start_t)
    print(f"|d1.Q| = {d1.Q}, |d2.q| = {d2.Q}, |d.Q| = {d.Q}")
    d.type = DFA.CONV_DFA

    results.append(
        Record(
            d1=d1.Q,
            d2=d2.Q,
            conv=d.Q,
            cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice),
            target=copy.deepcopy(d),
        )
    )

iter nr: 1
ex time with AS:  46.42249584197998
|d1.Q| = 35, |d2.q| = 35, |d.Q| = 1225
iter nr: 2
ex time with AS:  27.21225070953369
|d1.Q| = 26, |d2.q| = 23, |d.Q| = 598
iter nr: 3
ex time with AS:  240.68531036376953
|d1.Q| = 46, |d2.q| = 46, |d.Q| = 2116
iter nr: 4
ex time with AS:  42.57552194595337
|d1.Q| = 29, |d2.q| = 23, |d.Q| = 667
iter nr: 5
ex time with AS:  133.216144323349
|d1.Q| = 30, |d2.q| = 35, |d.Q| = 1050


In [15]:
results.sort(key=lambda x: x.conv)

# Remove degenerated cases (Conv DFA of size 1)
results = [r for r in results if r.conv != 1]

print(f"ilość wpisów w result: {len(results)}")
for i, r in enumerate(results):
    r.print_record()
    f = open("../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" + str(i) + ".txt", "w")
    f.write(results[i].get_complete_description())
    f.close()

ilość wpisów w result: 5
|d1| = 26, |d2| = 23, |conv| = 598, (mq, eq) = ([272604, 4], 141)
|d1| = 29, |d2| = 23, |conv| = 667, (mq, eq) = ([370929, 3], 164)
|d1| = 30, |d2| = 35, |conv| = 1050, (mq, eq) = ([811786, 5], 240)
|d1| = 35, |d2| = 35, |conv| = 1225, (mq, eq) = ([478522, 2], 114)
|d1| = 46, |d2| = 46, |conv| = 2116, (mq, eq) = ([1430182, 1], 208)


In [28]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)
    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    return df

In [ ]:
# EQ    : TTT     TTT with AS 
#       :      |
# DfaEx0: 583  | 14
# DfaEx1: 658  | 12
# DfaEx2: 1004 | 8
# DfaEx3: 1204 | 18
# DfaEx4: 2092 | 13

In [32]:
eqs = [583, 658, 1004, 1204, 2092]
eqs_as = [14, 12, 8, 18, 13]
red = ((np.array(eqs) - np.array(eqs_as)) / np.array(eqs)) * 100

columns = [('Target language', 'conv(DFA1, DFA2)'), ('Target language', 'DFA1'), ('Target language', 'DFA2'), ('TTT', 'EQ'), ('TTT with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [r.d1 for r in results],
    columns[2]: [r.d2 for r in results],
    columns[3]: [583, 658, 1004, 1204, 2092],
    columns[4]: [14, 12, 8, 18, 13],
    columns[5]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

Target language             TTT TTT with advice Reduction
  conv(DFA1, DFA2) DFA1 DFA2    EQ              EQ        EQ
0              598   26   23   583              14       97%
1              667   29   23   658              12       98%
2             1050   30   35  1004               8       99%
3             1225   35   35  1204              18       98%
4             2116   46   46  2092              13       99%